In [1]:
# preprocessing
import numpy as np 
import pandas as pd
import keras
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# Model
# from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization, GlobalAveragePooling2D, Flatten, MaxPooling2D, Concatenate, concatenate
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import TimeDistributed
from tensorflow.keras.optimizers import Nadam, Adam, SGD
# Save Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
# 繪製LOSS曲線
from matplotlib import pyplot as plt
import seaborn as sns
# Time
import time
import math
# Test
from sklearn.metrics import accuracy_score
import cv2

In [2]:
score_data = np.load('../input/deception/score_data.npy')

In [3]:
# stress = keras.models.load_model('../input/deception/mode35new_nohappydata.h5')

In [4]:
# def anger(p):
#     t = 0.343 * p + 1.003
#     return 2.332 * math.log(t)

# def fear(p):
#     t = 1.356 * p + 1
#     return 1.763 * math.log(t)

# def neutral(p):
#     t = 0.01229 * p + 1.036
#     return 5.03 * math.log(t)

# def disgust(p):
#     t = 0.0123 * p + 1.019
#     return 7.351 * math.log(t)

# def happiness(p):
#     t = 5.221e-5 * p + 0.9997
#     return 532.2 * math.log(t)

# def sad(p):
#     t = 0.1328 * p + 1.009
#     return 2.851 * math.log(t)

# def surprise(p):
#     t = 0.2825 * p + 1.003
#     return 2.478 * math.log(t)

In [5]:
# label_dict = {0: 'neutral', 1: 'happiness', 2: 'surprise', 3: 'sadness', 4: 'anger', 5: 'disgust', 6: 'fear'}
# emonamelst = []
# score_data = []
# plst = []
# for video in img_data[:]:
#     for frame in video:
#         frame = frame.astype(np.float32)
#         img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         img.resize(48,48)
#         # print(img.shape)
#         img = np.expand_dims(img, axis=0)  # makes image shape (1,48,48)
#         # print(img.shape)
#         img = img.reshape(1, 48, 48, 1)
#         # print(img.shape)
#         result = stress.predict(img)
#         result = list(result[0])
#         img_index = result.index(max(result))
#         p = max(result)
#         # print(label_dict[img_index])
#         emoname = label_dict[img_index]
#         emonamelst.append(emoname)
#         plst.append(p)
#         # print(p, emoname)
#         # print(emoname)
# #     print(emonamelst)
# #     print(plst)
#     add = 0
#     for i in range(15):
#         if emonamelst[i] == 'surprise':
#             add += 100 * surprise(plst[i])
#             #print(100 * self.surprise(plst[i]))
#             # print('addup:',add)
#         elif emonamelst[i] == 'sadness':
#             add += 100 * sad(plst[i])
#             # print(100 * self.sad(plst[i]))
#             # print('addup:',add)
#         elif emonamelst[i] == 'happiness':
#             add += 100 * happiness(plst[i])
#             # print(100 * self.happiness(plst[i]))
#             # print('addup:',add)
#         elif emonamelst[i] == 'disgust':
#             add += 100 * disgust(plst[i])
#             # print(100 * self.disgust(plst[i]))
#             # print('addup:',add)
#         elif emonamelst[i] == 'neutral':
#             add += 100 * neutral(plst[i])
#             # print(100 * self.neutral(plst[i]))
#             # print('addup:',add)
#         elif emonamelst[i] == 'fear':
#             add += 100 * fear(plst[i])
#             # print(100 * self.fear(plst[i]))
#             # print('addup:',add)
#         elif emonamelst[i] == 'anger':
#             add += 100 * anger(plst[i])
#             # print(100 * self.anger(plst[i]))
#     print('total add up:', add)
#     print('result of stress:', add / 15)
#     emonamelst = []
#     plst = []
#     score_data.append(add/15)
# from sklearn import preprocessing
# # score_data = preprocessing.scale(score_data)
# score_data = np.array(score_data)

In [6]:
img_data = np.load('../input/deception/img_data_new.npy')

In [7]:
labels = np.load('../input/deception/labels_new.npy')

In [8]:
len(score_data)

121

In [9]:
y = []
for label in labels:
  if label.split('_')[0]=='lie':
    y.append(0) # 0:lie
  else:
    y.append(1) # 1:truth

print(y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
num_classes=2 #one-hot encoding
Y = keras.utils.to_categorical(y, num_classes)

X_test, X_train, y_test, y_train = train_test_split(img_data, Y, test_size=0.7, shuffle=True, random_state=42)#訓練:測試
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size=0.5, shuffle=True, random_state=42)#從測試集再切出驗證集 測試:驗證=5:5

In [11]:
score_test,score_train = train_test_split(score_data, test_size=0.7, shuffle=True, random_state=42)
score_val, score_test = train_test_split(score_test,test_size=0.5, shuffle=True, random_state=42)

In [12]:
score_train.shape

(85,)

In [13]:
frames=15
channels=3
rows=128
columns=128
classes=2

'''!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_applications --no-deps
filename = "/opt/conda/lib/python3.7/site-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))
# VGGFace
import tensorflow as tf
from keras_vggface.vggface import VGGFace
video = Input(shape=(frames,rows,columns,channels))
cnn_base = VGGFace(input_shape=(rows,columns,channels),model="vgg16",include_top=False)

cnn_out = GlobalAveragePooling2D()(cnn_base.output)
# print(cnn_base.summary())
print("----------------------------------")
cnn = Model(inputs=cnn_base.input, outputs=cnn_out)
cnn.trainable = False'''


# InceptionResNetV2
video = Input(shape=(frames,rows,columns,channels))
cnn = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(128,128,3), pooling=None, classes=1000)
cnn_out = GlobalAveragePooling2D()(cnn.output)
cnn = Model(inputs=cnn.input, outputs=cnn_out)
cnn.trainable = False

2022-05-28 15:30:27.083883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 15:30:27.240692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 15:30:27.241716: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 15:30:27.243044: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

219070464/219055592 [==============================] - 1s 0us/step


In [14]:
cnn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 63, 63, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 63, 63, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 63, 63, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
# define LSTM model
model = Sequential()
model.add(TimeDistributed(cnn, input_shape=cnn.input.shape))
model.add(Dense(2048))
model.add(Dropout(0.3))
model.add(LSTM(1024))
#model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
#model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))


input_y = Input(shape=(1,))
y = Dense(1, activation="linear")(input_y)
y = Model(inputs=input_y, outputs=y)

combined = concatenate([model.output, y.output])
z = Dense(32, activation="relu")(combined)
z = Dense(32, activation="relu")(z)
z = Dense(2, activation="sigmoid")(z)

model_con = Model(inputs=[model.input, y.input], outputs=z)


opt = Nadam(learning_rate=0.000015) #learning_rate=0.00003
model_con.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
model_con.summary()

In [ ]:
callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    min_delta=0.0150,
    mode='auto',
    verbose=0,
    restore_best_weights=True
)

lrs = LearningRateScheduler(lambda epoch, lr: lr if epoch < 10 else lr/2)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=2, min_lr=0.0000001)

filepath = '/content/drive/MyDrive/畢專底嘉/程式碼/real life deception/checkpoint/'
checkpoint = keras.callbacks.ModelCheckpoint(filepath+'checkpoint_{epoch:02d}', save_freq='epoch')

checkpoint_filepath = 'model_stress.h5'
model_checkpoint= keras.callbacks.ModelCheckpoint(
          filepath=checkpoint_filepath,
          monitor='val_accuracy',
          mode='auto',
          save_best_only=True)

In [ ]:
from random import seed
import tensorflow
seed(157)
tensorflow.random.set_seed(157)

In [ ]:
score_train.shape

In [ ]:
class CustomSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch%10 == 0 :  # or save after some epoch, each k-th epoch etc.
            self.model.save("model_{}.h5".format(epoch))

saver = CustomSaver()

In [ ]:
history = model_con.fit([X_train,score_train], y_train, epochs=50, batch_size=65, validation_data=([X_val,score_val],y_val), callbacks=[saver]) #,checkpoint,callback,reduce_lr
fig, ax=plt.subplots(2,1,figsize=(12,10))
fig.suptitle('Train evaluation')

sns.lineplot(ax= ax[0],x=np.arange(0,len(history.history['accuracy'])),y=history.history['accuracy'])
sns.lineplot(ax= ax[0],x=np.arange(0,len(history.history['accuracy'])),y=history.history['val_accuracy'])

ax[0].legend(['Train','Validation'])
ax[0].set_title('Accuracy')

sns.lineplot(ax= ax[1],x=np.arange(0,len(history.history['loss'])),y=history.history['loss'])
sns.lineplot(ax= ax[1],x=np.arange(0,len(history.history['loss'])),y=history.history['val_loss'])

ax[1].legend(['Train','Validation'])
ax[1].set_title('Loss')
plt.show()

Y_pred=model_con.predict([X_test,score_test])
Y_pred = np.argmax(Y_pred, axis=1)
print('-Accuracy achieved on test set : {:.2f}%\n-Accuracy by model was: {:.2f}%\n-Accuracy by validation was: {:.2f}%'.
    format(accuracy_score(np.where(y_test == 1)[1],Y_pred)*100,(history.history['accuracy'][-1])*100,(history.history['val_accuracy'][-1])*100))

In [ ]:
'''Y_pred=model_con.predict([X_test,score_test])
print(Y_pred)
Y_pred = np.argmax(Y_pred, axis=1)
print(Y_pred)'''

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(np.where(y_test == 1)[1],Y_pred)
sns.set_style(style='dark')
plt.figure(figsize=(12,10))
heatmap = sns.heatmap(conf_mat,vmin=np.min(conf_mat.all()), vmax=np.max(conf_mat), annot=True,fmt='d', annot_kws={"fontsize":20},cmap='Spectral')
heatmap.set_title('Confusion Matrix Heatmap', fontdict={'fontsize':15}, pad=12)
heatmap.set_xlabel('Predicted',fontdict={'fontsize':14})
heatmap.set_ylabel('Actual',fontdict={'fontsize':14})
heatmap.set_xticklabels(['lie','truth'], fontdict={'fontsize':12})
heatmap.set_yticklabels(['lie','truth'], fontdict={'fontsize':12})
plt.show()

In [ ]:
# test = keras.models.load_model('./model_stress.h5')
# Y_pred=test.predict([X_test,score_test])
# Y_pred = np.argmax(Y_pred, axis=1)
# print(Y_pred)
# from sklearn.metrics import confusion_matrix
# conf_mat = confusion_matrix(np.where(y_test == 1)[1],Y_pred)
# sns.set_style(style='dark')
# plt.figure(figsize=(12,10))
# heatmap = sns.heatmap(conf_mat,vmin=np.min(conf_mat.all()), vmax=np.max(conf_mat), annot=True,fmt='d', annot_kws={"fontsize":20},cmap='Spectral')
# heatmap.set_title('Confusion Matrix Heatmap', fontdict={'fontsize':15}, pad=12)
# heatmap.set_xlabel('Predicted',fontdict={'fontsize':14})
# heatmap.set_ylabel('Actual',fontdict={'fontsize':14})
# heatmap.set_xticklabels(['lie','truth'], fontdict={'fontsize':12})
# heatmap.set_yticklabels(['lie','truth'], fontdict={'fontsize':12})
# plt.show()

In [ ]:
# Y_pred = test.predict([X_train,score_train])
# Y_pred = np.argmax(Y_pred, axis=1)
# print(accuracy_score(np.where(y_train == 1)[1],Y_pred))

In [ ]:
# Y_pred = test.predict([X_val,score_val])
# Y_pred = np.argmax(Y_pred, axis=1)
# print(accuracy_score(np.where(y_val == 1)[1],Y_pred))

In [ ]:
for i in range(0,91,10):
    test = keras.models.load_model('./model_{}.h5'.format(i))
    Y_pred = test.predict([X_test,score_test])
    Y_pred = np.argmax(Y_pred, axis=1)
    print('epoch: {}'.format(i),'\t{}'.format(accuracy_score(np.where(y_test == 1)[1],Y_pred)))